In [ ]:
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

from PIL import Image
from models import get_models
model_cls = get_models("qwen2_5_vl", logger)
model = model_cls(config={})

    # Prepare the input data
image_paths = [
    "./data/0b22fa63d0f54a529c525afbf2e8bb25_0.png",
    "./data/0b22fa63d0f54a529c525afbf2e8bb25_1.png",
    "./data/0b22fa63d0f54a529c525afbf2e8bb25_2.png",
    "./data/0b22fa63d0f54a529c525afbf2e8bb25_3.png",
]
images = []
for path in image_paths:
    try:
        img = Image.open(path).convert("RGB")
        images.append(img)
    except FileNotFoundError:
        logger.error(f"Image file not found: {path}")
        exit()

input_data = {
    "views": images,
    "instruction": "Describe the scene in detail."
}
prompt = "{image} {instruction}"

output = model(prompt, input_data)

print(output)

print("done")

In [1]:
import os
import pandas as pd
from datasets import Dataset, Features, Value, Image
from huggingface_hub import HfApi

# Mapping from direction string to column name
DIRECTION_MAP = {
    "left": "left",
    "front": "front",
    "right": "right",
    "back": "back"
}

def load_single_scan(scan_id, observation_dir):
    scan_folder = os.path.join(observation_dir)
    rows = {}

    for file in os.listdir(scan_folder):
        if not file.endswith(".png"):
            continue
        try:
            name = file[:-4]  # remove .png
            parts = name.split("_")
            if len(parts) < 3:
                print(f"Skipping malformed file name: {file}")
                continue
            scan_id_file = parts[0]
            viewpoint_id = parts[1]
            direction = "_".join(parts[2:])
        except Exception as e:
            print(f"Error parsing filename '{file}': {e}")
            continue

        if scan_id_file != scan_id or direction not in DIRECTION_MAP:
            continue

        key = (scan_id, viewpoint_id)
        if key not in rows:
            rows[key] = {
                "scan_id": scan_id,
                "viewpoint_id": viewpoint_id,
                "left": None,
                "front": None,
                "right": None,
                "back": None
            }

        rows[key][DIRECTION_MAP[direction]] = os.path.join(scan_folder, file)

    valid_rows = []
    for row in rows.values():
        if all(row[d] is not None for d in ["left", "front", "right", "back"]):
            valid_rows.append(row)
        else:
            print(f"Skipping incomplete viewpoint {row['viewpoint_id']} in scan {scan_id}")

    return pd.DataFrame(valid_rows) if valid_rows else None


/root/miniconda3/envs/navgpt/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
repo_id = "billzhao1030/MP3D_marked_obs"
api = HfApi()
api.create_repo(repo_id=repo_id, repo_type="dataset", exist_ok=True)

observation_dir = "../data/marked_obs"

features = Features({
    "scan_id": Value("string"),
    "viewpoint_id": Value("string"),
    "left": Image(),
    "front": Image(),
    "right": Image(),
    "back": Image()
})

scan_ids = list(set(f.split("_")[0] for f in os.listdir(observation_dir) if f.endswith(".png")))
scan_ids.sort()

for i, scan_id in enumerate(scan_ids):
    print(f"\n[{i+1}/{len(scan_ids)}] Processing scan: {scan_id}")
    df_scan = load_single_scan(scan_id, observation_dir)
    if df_scan is None:
        print(f"Skipping scan {scan_id} due to no valid data.")
        continue

    dataset = Dataset.from_pandas(df_scan, features=features)

    print(f"Pushing scan '{scan_id}' to Hugging Face Hub...")
    dataset.push_to_hub(repo_id, split=scan_id)
    print(f"✅ Uploaded split: {scan_id}")

print("\n🎉 Finished uploading all scans!")



[1/90] Processing scan: 17DRP5sb8fy
Pushing scan '17DRP5sb8fy' to Hugging Face Hub...


Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.35s/it]
No files have been modified since last commit. Skipping to prevent empty commit.


✅ Uploaded split: 17DRP5sb8fy

[2/90] Processing scan: 1LXtFkjw3qL
Pushing scan '1LXtFkjw3qL' to Hugging Face Hub...


Uploading the dataset shards: 100%|██████████| 1/1 [00:10<00:00, 10.48s/it]


✅ Uploaded split: 1LXtFkjw3qL

[3/90] Processing scan: 1pXnuDYAj8r
Pushing scan '1pXnuDYAj8r' to Hugging Face Hub...


...

✅ Uploaded split: x8F5xyUWy9e

[89/90] Processing scan: yqstnuAEVhm
Pushing scan 'yqstnuAEVhm' to Hugging Face Hub...


Uploading the dataset shards: 100%|██████████| 1/1 [00:03<00:00,  3.61s/it]


✅ Uploaded split: yqstnuAEVhm

[90/90] Processing scan: zsNo4HB9uLZ
Pushing scan 'zsNo4HB9uLZ' to Hugging Face Hub...


Uploading the dataset shards: 100%|██████████| 1/1 [00:02<00:00,  2.46s/it]


✅ Uploaded split: zsNo4HB9uLZ

🎉 Finished uploading all scans!
